# Artificial Intelligence
## Classifier
### Clothing

Basic classification: Classify images of clothing.

Classification is the process of predicting the class of given data points. Classes are sometimes called as targets/ labels or categories. Classification predictive modeling is the task of approximating a mapping function (f) from input variables (X) to discrete output variables (y).

Said another way...

A classifier in machine learning is an algorithm that automatically orders or categorizes data into one or more of a set of “classes.” The process of categorizing or classifying information based on certain characteristics is known as classification.

For example, spam detection in email service providers can be identified as a classification problem. This is s binary classification since there are only 2 classes as spam and not spam. A classifier utilizes some training data to understand how given input variables relate to the class. In this case, known spam and non-spam emails have to be used as the training data. When the classifier is trained accurately, it can be used to detect an unknown email.

Classification belongs to the category of supervised learning where the targets also provided with the input data. There are many applications in classification in many domains such as in credit approval, medical diagnosis, target marketing etc.


Types of Classifiers:
+ Binary Classifiers: These are used when there are only two possible classes. For example, an email classifier might be designed to detect spam and non-spam emails.
+ Multiclass Classifiers: These handle situations where there are more than two classes. For instance, a classifier that categorizes news articles into topics like sports, politics, and technology.
+ Multilabel Classifiers: These can assign multiple labels to each instance. For example, a movie could be classified into multiple genres like comedy,drama, and action simultaneously.



### Reference:
+ https://www.tensorflow.org/tutorials/keras/classification
+ https://towardsdatascience.com/machine-learning-classifiers-a5cc4e1b0623

In [ ]:
PROJECT_NAME="ai-training-2024-08-09-bucket"

## Environment Check

In [ ]:
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#- Google Colab Check
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
RunningInCOLAB = False
RunningInCOLAB = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    print("You are running this notebook in Google Colab.")
else:
    print("You are running this notebook with Jupyter iPython runtime.")

## Library Management

In [ ]:
############################################
# INCLUDES
############################################
#libraries specific to this example
## Imports
import tensorflow as tf
from tensorflow import keras

import glob
import imageio
import matplotlib as matplt
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time

from IPython import display

#a set of libraries that perhaps should always be in Python source
import os
import datetime
import sys
import gc
import getopt
import inspect
import math
import warnings

#a darn useful library for creating paths and one I recommend you load to your environment
from pathlib import Path

from pydoc import help                          # can type in the python console `help(name of function)` to get the documentation

warnings.filterwarnings('ignore')               # don't print out warnings

## Variables

In [ ]:
############################################
# GLOBAL VARIABLES
############################################
DEBUG = 1
DEBUG_DATA = 0

# CODE CONSTRAINTS
VERSION_NAME    = "MLCLASSIFIER"
VERSION_MAJOR   = 0
VERSION_MINOR   = 0
VERSION_RELEASE = 1

#used for values outside standard ASCII, just do it, you'll need it
ENCODING  ="utf-8"

############################################
# GLOBAL CONSTANTS
############################################
BUFFER_SIZE = 60000
BATCH_SIZE = 256

############################################
# APPLICATION VARIABLES
############################################

############################################
# GLOBAL CONFIGURATION
############################################
os.environ['PYTHONIOENCODING']=ENCODING


## Function

In [ ]:
## Outputs library version history of effort.
#
def lib_diagnostics() -> None:

    import pkg_resources

    package_name_length=40
    package_version_length=20

    # Get installed packages
    the_packages=["nltk", "numpy", "os", "pandas"]
    installed = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
    for package_idx, package_name in enumerate(installed):
         if package_name in the_packages:
             installed_version = installed[package_name]
             print(f"{package_name:<40}#: {str(pkg_resources.parse_version(installed_version)):<20}")

    try:
        print(f"{'TensorFlow version':<40}#: {str(tf.__version__):<20}")
        print(f"{'     gpu.count:':<40}#: {str(len(tf.config.experimental.list_physical_devices('GPU')))}")
        print(f"{'     cpu.count:':<40}#: {str(len(tf.config.experimental.list_physical_devices('CPU')))}")
    except Exception as e:
        pass

    try:
        print(f"{'Torch version':<40}#: {str(torch.__version__):<20}")
        print(f"{'     GPUs available?':<40}#: {torch.cuda.is_available()}")
        print(f"{'     count':<40}#: {torch.cuda.device_count()}")
        print(f"{'     current':<40}#: {torch.cuda.current_device()}")
    except Exception as e:
        pass


    try:
      print(f"{'OpenAI Azure Version':<40}#: {str(the_openai_version):<20}")
    except Exception as e:
      pass
    return

## Function Call

In [ ]:
lib_diagnostics()

## Input Sources

### Load and prepare the dataset

You will use the MNIST dataset to obtain data.

*Note*:
MNIST database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems.  Reference: https://en.wikipedia.org/wiki/MNIST_database

This example uses the Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

Fashion MNIST is intended as a drop-in replacement for the classic MNIST dataset—often used as the "Hello, World" of machine learning programs for computer vision. The MNIST dataset contains images of handwritten digits (0, 1, 2, etc.) in a format identical to that of the articles of clothing you'll use here.

This guide uses Fashion MNIST for variety, and because it's a slightly more challenging problem than regular MNIST. Both datasets are relatively small and are used to verify that an algorithm works as expected. They're good starting points to test and debug code.

Here, 60,000 images are used to train the network and 10,000 images to evaluate how accurately the network learned to classify images. You can access the Fashion MNIST directly from TensorFlow.

In [ ]:
#create reference to dataset
fashion_mnist = keras.datasets.fashion_mnist

#load the data and split into train/test datasets
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
#Train variable details
print("Your training data:")
print("{} shape is {}.".format("Train images ", train_images.shape) )
print("{} shape is {}.".format("Train labels ", train_labels.shape) )
print("{} sample structure is {}".format("Train labels", train_labels))
print("These images and thier corresponding labels represent trusted data to train the model on.")
print("")

#Test variable details
print("Your test data:")
print("{} shape is {}.".format("Test images ", test_images.shape) )
print("{} shape is {}.".format("Test labels ", test_labels.shape) )
print("{} sample structure is {}".format("Test labels", test_labels))
print("These images and thier corresponding labels represent trusted data to test the model on.  How does the model know if it's performing well?")
print("By using the test data per iteration to check itself.")

### Dataset Definition

The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. The labels are an array of integers, ranging from 0 to 9. These correspond to the class of clothing the image represents:

Label 	Class

+ 0 	T-shirt/top
+ 1 	Trouser
+ 2 	Pullover
+ 3 	Dress
+ 4 	Coat
+ 5 	Sandal
+ 6 	Shirt
+ 7 	Sneaker
+ 8 	Bag
+ 9 	Ankle boot

Each image is mapped to a single label. Since the class names are not included with the dataset, store them here to use later when plotting the images:

In [ ]:
#associate "human legible" labels to the actual dataset
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Example Image from Dataset

In [ ]:
#show the first image from the training dataset
plt.figure()
plt.imshow(train_images[1])
plt.colorbar()
plt.grid(False)
plt.show()

## Preprocess the data

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255:

Scale these values to a range of 0 to 1 before feeding them to the neural network model. To do so, divide the values by 255. It's important that the training set and the testing set be preprocessed in the same way.  Ultimately your inference / prediction of data will follow the same method of scaling.

If you don't make your inference inputs align to the same method(s) you used during training, you cannot expect the same level of quality result obtained during training.

In [ ]:
#scale the images between 0 - 1
train_images = train_images / 255.0
test_images = test_images / 255.0

## Show Sample Data
Show the first 25 images from the training dataset and display the classification name with each.

In [ ]:
#sneak peek of the data
#Did you notice a difference in the output of the data?  What's the difference between the first image seen and these images?
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

### Set up the layers

The basic building block of a neural network is the layer. Layers extract representations from the data fed into them.

Most of deep learning consists of chaining together simple layers. Most layers, such as tf.keras.layers.Dense, have parameters that are learned during training.

Rectified linear unit (ReLU), is an activation function which means, this is how the neuron determines if it's going to fire (weighting of probabilities). See: https://en.wikipedia.org/wiki/Rectifier_(neural_networks) The short version is that ReLU is considered an excellent tool and using a vanishing gradient to determine probabilities always making sure any value less than zero is set to 0.

Your output layer must equal the number of questions you're seeking to ask.  In this case we have ten (10) categories of clothing so we need 10 neurons to represent those different potential answers.

Notice that the "shape" of the first layer corresponds to the shape of the input image.

In [ ]:
#capture the function calls return into a variable, this encapsulate a complex object that defines the entire neural model
model = keras.Sequential([
    #one-D array are preferred for all processing
    #The first layer in this network, tf.keras.layers.Flatten, transforms the format of the images from a two-dimensional array
    #(of 28 by 28 pixels) to a one-dimensional array (of 28 * 28 = 784 pixels). Think of this layer as unstacking rows of pixels
    #in the image and lining them up. This layer has no parameters to learn; it only reformats the data.
    keras.layers.Flatten(input_shape=(28, 28)),

    #first layer is 128 neurons and a ReLU activation function.
    keras.layers.Dense(128, activation='relu'),

    #output layer defined
    keras.layers.Dense(10)
])

## Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

+ Loss function —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.

+ Optimizer —This is how the model is updated based on the data it sees and its loss function.

+ Metrics —Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.


In [ ]:
model.compile(optimizer='adam',
              #v2.1 accepts this activation method but previous versions DO NOT
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


## Train the model

### Training the neural network model requires the following steps:

    1. Feed the training data to the model. In this example, the training data is in the train_images and train_labels arrays.
    2. The model learns to associate images and labels.
    3. You ask the model to make predictions about a test set—in this example, the test_images array.
    4. Verify that the predictions match the labels from the test_labels array.

## Train the model

### Training the neural network model requires the following steps:

    1. Feed the training data to the model. In this example, the training data is in the train_images and train_labels arrays.
    2. The model learns to associate images and labels.
    3. You ask the model to make predictions about a test set—in this example, the test_images array.
    4. Verify that the predictions match the labels from the test_labels array.

In [ ]:
#given the images and labels begin learning from the train_* dataset for 10 "iterations".
#note that you are ACTIVELY training a neural layer (AI) solution now.
model.fit(train_images, train_labels, epochs=10)

## Epochs

Notice each epoc completion demonstrates an accuracy improvement.  You can add a *patience* function to drop out when your training reaches an optimal level to avoid overfitting.  Additional *drop out* functions are available as well.

***Ask the instructor what other drop out functions are available.***

## Evaluate accuracy

Now we see how well the model performs.  The `evaluate()` function performs a prediction by taking the test_* inputs and and performing a diff between actual data and predicted data.

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

As previously discussed overfitting is a concern.  Notes from the demonstration code:

It turns out that the accuracy on the test dataset is a little less than the accuracy on the training dataset. This gap between training accuracy and test accuracy represents overfitting. Overfitting happens when a machine learning model performs worse on new, previously unseen inputs than it does on the training data. An overfitted model "memorizes" the noise and details in the training dataset to a point where it negatively impacts the performance of the model on the new data. For more information, see the following:

1.  Demonstration of overfitting - https://www.tensorflow.org/tutorials/keras/overfit_and_underfit#demonstrate_overfitting
2.  Strategies to prevent overfitting - https://www.tensorflow.org/tutorials/keras/overfit_and_underfit#strategies_to_prevent_overfitting

## Make Predictions

With the model trained, you can use it to make predictions about some images. The model's linear outputs, logits. Attach a softmax layer to convert the logits to probabilities, which are easier to interpret.

### Logits

The vector of raw (non-normalized) predictions that a classification model generates, which is ordinarily then passed to a normalization function. If the model is solving a multi-class classification problem, logits typically become an input to the softmax function. The softmax function then generates a vector of (normalized) probabilities with one value for each possible class.

In addition, logits sometimes refer to the element-wise inverse of the sigmoid function. For more information, see tf.nn.sigmoid_cross_entropy_with_logits.

In [ ]:
#define another model designed to help identify classes
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])

#given the test images what labels have precipitated out?
predictions = probability_model.predict(test_images)

In [ ]:
#let's see what the predictions resulting data structure looks like on the inside.
predictions

#the resultant is an array of arrays [][]
# each row represents each image predicted against
#   each column in each row are the probabilities for each "class"

#let's see what the predictions resulting data structure looks like on the inside.

Predictions for the first input (image) submitted.

In [ ]:
predictions[0]

In [ ]:
#numpy magic, obtain the highest value of the array
the_answer=np.argmax(predictions[0])

#so for the first test image the most likely candidate for a class is element 9 in the array (remember, arrays go from 0..9)
#what does element 9 in the classes array defined earlier represent?  what class?

So, the model is most confident that this image is an ankle boot, or class_names[9]. Examining the test label shows that this classification is correct:

In [ ]:
print("Label {} relates to a {}.".format(the_answer,class_names[np.argmax(predictions[0])]))

Graph this to look at the full set of 10 class predictions.

In [ ]:
#functions defined to show the item and relevance of the prediction

def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')


Let's plot several images with their predictions. Note that the model can be wrong even when very confident.

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()


## Use the trained model

Use the trained model to make a prediction about a single image.  We will generate a random number selected out of the domain of values.

### *Note*
tf.keras models are optimized to make predictions on a batch, or collection, of examples at once. Accordingly, even though you're using a single image, you need to add it to a list:


In [ ]:
# seed the pseudorandom number generator
from random import seed
from random import random

# seed random number generator
seed(1)

#lowest array position
min=0
#highest array position (note that the returned shape is 1000x28x28)
max=test_images.shape[0]-1

#generate a random value
value = random()

#scale that value to the range we have available
scaled_value = min + (value * (max - min))
scaled_value = int(scaled_value)


#take the an image from the test dataset, normally a 28x28 matrix
img = test_images[scaled_value]

# Add the image to a batch where it's the only member, now a 1x28x28 matrix.
img = (np.expand_dims(img,0))

#predict the correct label for this image
predictions_single = probability_model.predict(img)

print(predictions_single)

In [ ]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

In [ ]:
#keras.Model.predict returns a list of lists—one list for each image in the batch of data. Grab the predictions for our (only) image in the batch
np.argmax(predictions_single[0])

### Which means...

In [ ]:
print("Prediction is label {} which is a {}.".format(np.argmax(predictions_single[0]),class_names[np.argmax(predictions_single[0])]))
print("Reality suggests label {} which is a {}.".format(test_labels[scaled_value],class_names[test_labels[scaled_value]] ))

In [ ]:
plt.figure(figsize=(1,1))
plot_image(scaled_value,np.argmax(predictions_single[0]), test_labels, test_images)
plt.tight_layout()
plt.show()


In [ ]:
#let's try another one
#generate a random value
value = random()

#scale that value to the range we have available
scaled_value = min + (value * (max - min))
scaled_value = int(scaled_value)
#take the an image from the test dataset, normally a 28x28 matrix
img = test_images[scaled_value]

# Add the image to a batch where it's the only member, now a 1x28x28 matrix.
img = (np.expand_dims(img,0))
#predict the correct label for this image
predictions_single = probability_model.predict(img)

print(predictions_single)
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)
np.argmax(predictions_single[0])
print("Prediction is label {} which is a {}.".format(np.argmax(predictions_single[0]),class_names[np.argmax(predictions_single[0])]))
print("Reality suggests label {} which is a {}.".format(test_labels[scaled_value],class_names[test_labels[scaled_value]] ))
plt.figure(figsize=(1,1))
plot_image(scaled_value,np.argmax(predictions_single[0]), test_labels, test_images)
plt.tight_layout()
plt.show()